In [1]:
from pathlib import Path
import numpy as np
import xarray as xr

In [2]:
dloc = Path("/Volumes/Jedha/CMIP6")


f2 = dloc / 'ta_Amon_CNRM-CM6-1_amip-lwoff_r1i1p1f2_gr_197901-201412.nc'
f1 = dloc / 'ta_Amon_CNRM-CM6-1_amip_r1i1p1f2_gr_197901-201412.nc'

ds1 = xr.open_dataset(f1)
ds2 = xr.open_dataset(f2)

t1 = ds1['ta']
t2 = ds2['ta']

In [31]:

psds1 = xr.open_dataset(dloc / 'ps_Amon_CNRM-CM6-1_amip-lwoff_r1i1p1f2_gr_197901-201412.nc')
psds2 = xr.open_dataset(dloc / 'ps_Amon_CNRM-CM6-1_amip_r1i1p1f2_gr_197901-201412.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/Volumes/Jedha/CMIP6/ps_Amon_CNRM-CM6-1_amip-lwoff_r1i1p1f2_gr_197901-201412.nc'

In [15]:
# Goff-Gratch WMO 1987
# http://cms.ncas.ac.uk/documents/vn4.5/p029.pdf
def esat_ice(T):
    triple = 273.16
    return np.exp(-9.09685*(triple/T - 1) 
                  - 3.56654*np.log(triple/T) 
                  + 0.87682*(1 - T/triple)
                  + 0.78614 + 2.0)

def esat_liq(T):
    triple = 273.16
    return np.exp(10.79574 * (1 - triple/T)
                 - 5.028*np.log(T/triple)
                 + 1.50475e-4 * (1 - 10**(-8.2969*(T/triple - 1)))
                 + 0.42873e-3 * (10**(4.76955*(1-triple/T)) - 1)
                 + 0.78614 + 2.0)

def esat(T):
    ei = esat_ice(T)
    ew = esat_liq(T)
    return np.where(T < 273.15, ei, ew)


def qsat(T, p):
    """
    T in Kelvin 
    p in Pa
    """
    epsilon = 0.62198 # ratio of molecular weights of water and dry air
    return epsilon * esat(T) / p

In [13]:
t1

<xarray.DataArray 'ta' (time: 432, plev: 19, lat: 128, lon: 256)>
array([[[[      nan, ...,       nan],
         ...,
         [252.6984 , ..., 252.6984 ]],

        ...,

        [[288.0589 , ..., 288.0589 ],
         ...,
         [261.67526, ..., 261.67526]]],


       ...,


       [[[      nan, ...,       nan],
         ...,
         [254.12881, ..., 254.12881]],

        ...,

        [[285.24896, ..., 285.24896],
         ...,
         [238.15854, ..., 238.15854]]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon      (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
  * plev     (plev) float32 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * time     (time) datetime64[ns] 1979-01-16T12:00:00 ... 2014-12-16T12:00:00
Attributes:
    long_name:           Air Temperature
    units:               K
    online_operation:    average
    cell_methods:        time: mean
    interval_operation:  900 s
    interval_write:      1 month
    standard_name:       air_temperature
    description:         Air Temperature
    history:             none
    cell_measures:       area: areacella

In [23]:
sat_spec_hum_1 = t1.sel(plev=slice(None, 10000.)).copy()
# print(sat_spec_hum_1)
for k, p in enumerate(sat_spec_hum_1['plev']):
    tmp = qsat(t1.isel(plev=k).squeeze(), p.values)
    print(tmp.shape)
    sat_spec_hum_1[:, k, :, :] = tmp

(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)


In [24]:
sat_spec_hum_2 = t2.sel(plev=slice(None, 10000.)).copy()
# print(sat_spec_hum_1)
for k, p in enumerate(sat_spec_hum_2['plev']):
    tmp = qsat(t2.isel(plev=k).squeeze(), p.values)
    print(tmp.shape)
    sat_spec_hum_2[:, k, :, :] = tmp

(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)
(432, 128, 256)


In [28]:
def vertical_integral(field):
    # now we have to vertically integrate that q*
    gravity = 9.81
    # reverse the lev dimension
    field = field[:,::-1,:,:]
    delp = np.diff(field['plev'])
    return (delp*field).sum(axis=1)/gravity


In [29]:
column_sat_spec_hum_2 = vertical_integral(sat_spec_hum_2)

ValueError: operands could not be broadcast together with shapes (11,) (432,12,128,256) 

In [30]:
t1['plev']

<xarray.DataArray 'plev' (plev: 19)>
array([100000.,  92500.,  85000.,  70000.,  60000.,  50000.,  40000.,  30000.,
        25000.,  20000.,  15000.,  10000.,   7000.,   5000.,   3000.,   2000.,
         1000.,    500.,    100.], dtype=float32)
Coordinates:
  * plev     (plev) float32 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
Attributes:
    name:           plev
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down